# Translation Agent using LangGraph and Google Gemini API

This notebook demonstrates a translation agent that translates text to multiple languages using:
- **Google Gemini API** for translation
- **LangGraph** for workflow orchestration

## Supported Languages:
1. English (US)
2. English (Australia)
3. Vietnamese
4. Thai
5. Hindi


In [1]:
# Install required packages
%pip install -q langchain-google-genai langgraph langchain-core



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from typing import TypedDict, Annotated
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage

# Set your Google Gemini API key
# You can set it as an environment variable or replace the string below
API_KEY = os.getenv("GOOGLE_API_KEY", "your-api-key")

# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=API_KEY,
    temperature=0.3
)


## Define State and Translation Nodes


In [3]:
# Define a reducer function to merge dictionaries
def merge_dicts(left: dict, right: dict) -> dict:
    """Merge two dictionaries, with right taking precedence"""
    # Handle None values
    if left is None:
        left = {}
    if right is None:
        right = {}
    # Merge dictionaries, with right taking precedence
    return {**left, **right}

# Define the state for our translation graph
class TranslationState(TypedDict):
    original_text: str
    translations: Annotated[dict, merge_dicts]
    current_language: str

# Define target languages with their proper names for the API
TARGET_LANGUAGES = {
    "en-US": "English (US)",
    "en-AU": "English (Australia)",
    "vi": "Vietnamese",
    "th": "Thai",
    "hi": "Hindi"
}


In [4]:
def translate_text(state: TranslationState) -> TranslationState:
    """Translate text to the current target language"""
    current_lang = state["current_language"]
    lang_name = TARGET_LANGUAGES[current_lang]
    original_text = state["original_text"]
    
    # Create translation prompt
    prompt = f"""Translate the following text to {lang_name}. 
Return only the translation, no explanations or additional text.

Original text: {original_text}

Translation ({lang_name}):"""
    
    # Get translation from Gemini
    response = llm.invoke([HumanMessage(content=prompt)])
    translation = response.content.strip()
    
    # Update state with translation
    return {
        "translations": {current_lang: translation}
    }


In [5]:
def create_translation_node(language_code: str):
    """Create a translation node for a specific language"""
    def node_func(state: TranslationState) -> TranslationState:
        updated_state = state.copy()
        updated_state["current_language"] = language_code
        result = translate_text(updated_state)
        return result
    return node_func


## Build the LangGraph Workflow


In [6]:
# Create the graph
workflow = StateGraph(TranslationState)

# Add nodes for each target language
for lang_code in TARGET_LANGUAGES.keys():
    workflow.add_node(lang_code, create_translation_node(lang_code))

# Set entry point
workflow.set_entry_point("en-US")

# Add edges to chain translations
workflow.add_edge("en-US", "en-AU")
workflow.add_edge("en-AU", "vi")
workflow.add_edge("vi", "th")
workflow.add_edge("th", "hi")
workflow.add_edge("hi", END)

# Compile the graph
app = workflow.compile()


## Execute Translation


In [7]:
# Input text to translate
input_text = "Hello, how are you today? I hope you're doing well."

# Initial state
initial_state = {
    "original_text": input_text,
    "translations": {},
    "current_language": "en-US"
}

# Run the translation workflow
result = app.invoke(initial_state)

# Print results
print("=" * 80)
print("TRANSLATION RESULTS")
print("=" * 80)
print(f"\nOriginal Text: {input_text}\n")
print("-" * 80)

for lang_code, lang_name in TARGET_LANGUAGES.items():
    if lang_code in result["translations"]:
        print(f"{lang_name} ({lang_code}):")
        print(f"  {result['translations'][lang_code]}")
        print()


TRANSLATION RESULTS

Original Text: Hello, how are you today? I hope you're doing well.

--------------------------------------------------------------------------------
English (US) (en-US):
  Hello, how are you today? I hope you're doing well.

English (Australia) (en-AU):
  G'day, how ya goin' today? Hope you're doing well.

Vietnamese (vi):
  Chào bạn, hôm nay bạn thế nào? Mong bạn luôn khỏe.

Thai (th):
  สวัสดีครับ วันนี้สบายดีไหมครับ? หวังว่าคุณสบายดีนะครับ

Hindi (hi):
  नमस्ते, आप आज कैसे हैं? मुझे उम्मीद है कि आप ठीक होंगे।



## Export Results to Excel

Export translation results to an Excel file for easy sharing and analysis.


In [8]:
# Install required packages for Excel export
%pip install -q pandas openpyxl



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
from datetime import datetime

def export_translations_to_excel(result: dict, original_text: str, filename: str = None):
    """
    Export translation results to an Excel file with languages as column headers.
    
    Args:
        result: The result dictionary from the translation workflow
        original_text: The original text that was translated
        filename: Optional filename. If not provided, generates a timestamped filename.
    """
    # Generate filename if not provided
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"translations_{timestamp}.xlsx"
    
    # Prepare data dictionary with Original Text first, then each language as a column
    data = {"Original Text": [original_text]}
    
    # Add each language translation as a column
    for lang_code, lang_name in TARGET_LANGUAGES.items():
        if lang_code in result.get("translations", {}):
            data[lang_name] = [result["translations"][lang_code]]
        else:
            data[lang_name] = [""]  # Empty string if translation not available
    
    # Create DataFrame with a single row
    df = pd.DataFrame(data)
    
    # Export to Excel
    df.to_excel(filename, index=False, sheet_name="Translations")
    
    print(f"✓ Translations exported to: {filename}")
    print(f"  Format: {len(data)} columns, 1 row")
    
    return filename

# Export the latest translation results
if 'result' in locals():
    excel_file = export_translations_to_excel(result, input_text)
    print(f"\nExcel file created: {excel_file}")
else:
    print("Run the translation workflow first to generate results.")


✓ Translations exported to: translations_20251105_212719.xlsx
  Format: 6 columns, 1 row

Excel file created: translations_20251105_212719.xlsx


## Batch Translate from Excel

Provide an input Excel file with a column containing the text to translate. This section will iterate each row, translate to all target languages, and write a new Excel with results.


In [13]:
import pandas as pd
from datetime import datetime
from typing import Optional


def translate_dataframe_from_excel(
    input_excel_path: str,
    text_column: str = "Original Text",
    output_excel_path: Optional[str] = None,
    batch_size: int = 10,
    verbose: bool = True,
):
    """
    Read an Excel file, translate each row's text to all target languages, and save results to a new Excel.

    Args:
        input_excel_path: Path to input Excel file.
        text_column: Name of the column with source text to translate.
        output_excel_path: Optional path for the output Excel; timestamped if not provided.
        batch_size: Number of rows between progress prints.
        verbose: Whether to print progress.

    Returns:
        Path to the written Excel file.
    """
    # Load input
    df_in = pd.read_excel(input_excel_path)
    if text_column not in df_in.columns:
        raise ValueError(
            f"Column '{text_column}' not found in input. Available columns: {list(df_in.columns)}"
        )

    # Prepare output columns
    out_columns = ["Original Text"] + [TARGET_LANGUAGES[lc] for lc in TARGET_LANGUAGES]
    rows_out = []

    num_rows = len(df_in)
    if verbose:
        print(f"Found {num_rows} rows. Starting translation...")

    # Iterate and translate
    for idx, row in df_in.iterrows():
        original_text = str(row[text_column]) if pd.notna(row[text_column]) else ""
        if original_text.strip() == "":
            # Keep empty row with blanks
            out_row = {col: "" for col in out_columns}
            out_row["Original Text"] = original_text
            rows_out.append(out_row)
            continue

        initial_state = {
            "original_text": original_text,
            "translations": {},
            "current_language": "en-US",
        }

        try:
            result_row = app.invoke(initial_state)
            out_row = {col: "" for col in out_columns}
            out_row["Original Text"] = original_text
            for lang_code, lang_name in TARGET_LANGUAGES.items():
                out_row[lang_name] = result_row["translations"].get(lang_code, "")
        except Exception as e:
            out_row = {col: "" for col in out_columns}
            out_row["Original Text"] = original_text
            # Put error note in first language column to surface failure
            first_lang_name = TARGET_LANGUAGES[next(iter(TARGET_LANGUAGES.keys()))]
            out_row[first_lang_name] = f"ERROR: {e}"

        rows_out.append(out_row)

        if verbose and (idx + 1) % batch_size == 0:
            print(f"Translated {idx + 1}/{num_rows} rows...")

    # Build DataFrame
    df_out = pd.DataFrame(rows_out, columns=out_columns)

    # Determine output path
    if output_excel_path is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_excel_path = f"translations_{timestamp}.xlsx"

    # Write Excel
    df_out.to_excel(output_excel_path, index=False, sheet_name="Translations")

    if verbose:
        print(f"✓ Wrote {len(df_out)} rows to {output_excel_path}")

    return output_excel_path


# Example usage (adjust paths/column as needed):
output_path = translate_dataframe_from_excel(
    input_excel_path="./input.xlsx",
    text_column="Original Text",  # or your column name
)
output_path


Found 3 rows. Starting translation...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10
Please retry in 3.615858645s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, retry_delay {
  seco

✓ Wrote 3 rows to translations_20251105_215206.xlsx


'translations_20251105_215206.xlsx'